* A modelagem irá seguir a arquitetura dos transformers utilizando o Bert multilingual para conseguir inferir todos os idiomas presentes no dataset, todas as informações da arquitetura e treinamento podem ser observadas no CONFIG. Para avaliação o dataset foi divido em folds aplicando o crossvalidation e o melhor modelo seleciona é o que apresenta menor loss sendo esse utilizado na fase de validação

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AdamW

model_name1 = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name1, )

CONFIG = {"seed": 42, 
          "epochs": 3, 
          "model_name": model_name1,
          "train_batch_size": 20,
          "valid_batch_size": 2,
          "max_length": 256,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 2, 
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": 'model'
          }

In [2]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_listing = pd.read_csv('listings.csv')
df_listing = df_listing[['id','name','description','neighborhood_overview','review_scores_rating']]
df_listing = df_listing.rename(columns={'id':'reviewer_id'})
df_reviews = pd.read_csv('reviews.csv')
df = df_reviews.merge(df_listing, left_on='listing_id', right_on='reviewer_id')

#df = pd.read_csv('train.csv')
df['comments'] = df['comments'].astype(str)
#df['review_scores_rating'] = df['review_scores_rating'].astype(float)

In [3]:
df['comments'] = df['comments'].astype(str)

In [4]:
import langid
from tqdm.notebook import tqdm
idoms = []

for i in tqdm(df['comments']):
    a, b = langid.classify(str(i))
    idoms.append(a)

In [5]:
df['idioms'] = idoms
df.head()

,listing_id,id,date,reviewer_id_x,reviewer_name,comments,reviewer_id_y,name,description,neighborhood_overview,review_scores_rating,idioms
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en


In [6]:
df.head()

,listing_id,id,date,reviewer_id_x,reviewer_name,comments,reviewer_id_y,name,description,neighborhood_overview,review_scores_rating,idioms
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

class ReviewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = self.df['comments'].values
        self.y = df['review_scores_rating'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        y = self.y[index]
        
        inputs_tokenizer = self.tokenizer(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        inputs_ids = inputs_tokenizer['input_ids']
        inputs_mask = inputs_tokenizer['attention_mask']
           
        return {
            'inputs_ids': torch.tensor(inputs_ids, dtype=torch.long),
            'inputs_mask': torch.tensor(inputs_mask, dtype=torch.long),
            'value': torch.tensor(y, dtype=torch.float)
            
        }
    
class Net(nn.Module):
    def __init__(self, model_name):
        super(Net, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, 1)
        
    def forward(self, ids, mask):        
        
        out = self.model(input_ids=ids,
                         attention_mask=mask,
                         output_hidden_states=False)
            
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

def criterion(outputs1, outputs2):  
    list_a = []
    list_b = []
    
    outputs1 = torch.flatten(outputs1)
    for o in outputs2:
        list_b.append(o.item())

    loss = nn.MSELoss()
    return loss(outputs1.to(CONFIG['device']), torch.Tensor(list_b).to(CONFIG['device']))

In [8]:
import tqdm.notebook as tqdm
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    print(dataloader)
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    
    for step, data in bar:
        
        input_ids = data['inputs_ids'].to(device, dtype = torch.long)
        inputs_mask = data['inputs_mask'].to(device, dtype = torch.long)
 
        value = data['value'].to(device)
        batch_size = input_ids.size(0)

        result = model(input_ids, inputs_mask)
        
        loss = criterion(result, value)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss


@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        data_toxic_ids = data['inputs_ids'].to(device, dtype = torch.long)
        data_toxic_mask = data['inputs_mask'].to(device, dtype = torch.long)
        value = data['value'].to(device)
        batch_size = data_toxic_ids.size(0)
        result = model(data_toxic_ids, data_toxic_mask)
        
        loss = criterion(result, value)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [9]:
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = ReviewsDataset(df_train, tokenizer=tokenizer, max_length=CONFIG['max_length'])
    valid_dataset = ReviewsDataset(df_valid, tokenizer=tokenizer, max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [10]:
from tqdm.notebook import tqdm
from collections import defaultdict
import wandb

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

In [11]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df['idioms'])):
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
df.head()

/home/josu/workspace38/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


,listing_id,id,date,reviewer_id_x,reviewer_name,comments,reviewer_id_y,name,description,neighborhood_overview,review_scores_rating,idioms,kfold
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en,0
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en,0
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en,1
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en,1
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...,17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,4.68,en,1


In [12]:
import time
import copy
import numpy as np
import gc
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    run = wandb.init(project='Cognitivo', 
                     config=CONFIG,
                     job_type='Train',
                     anonymous='must')
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = Net(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    
    run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()

====== Fold: 0 ======


wandb: Currently logged in as: anony-moose-172447 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: NVIDIA GeForce RTX 3060




Validation Loss Improved (inf ---> 0.08607730353938059)
Model Saved



...

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: NVIDIA GeForce RTX 3060




Validation Loss Improved (inf ---> 0.13615218526081727)
Model Saved



RuntimeError: CUDA out of memory. Tried to allocate 60.00 MiB (GPU 0; 11.77 GiB total capacity; 8.48 GiB already allocated; 13.50 MiB free; 9.01 GiB reserved in total by PyTorch)